# CORDIS SDG EDA

Perform and write up initial analysis of CORDIS projects labelled with SDGs

QD
- [x] Validation of classification by manual sampling of projects labelled with SDGs
- [ ] [On hold?] Top terms associated with SDGs
- [x] Percentage of projects that had classification successfully performed

EDA
- [x] Change in SDG activity over time
- [x] Change in funding for SDGs over time
- [x] SDG activity and specialisation by coordinator country
- [ ] SDG activity and specialisation by participant countries
- [ ] SDG activity and specialisation by country involvement

Extra Ideas
- [ ] Diversity of countries by SDG
- [ ] Diversity of SDGs by country

In [ ]:
%run ../notebook_preamble.ipy

In [ ]:
from sdg_mapping.cordis import load_cordis_projects, load_cordis_project_sdgs
from sdg_mapping.cordis.cordis_utils import FRAMEWORK_PROGRAMMES
from sdg_mapping.utils.sdg_utils import sdg_hex_color_codes, sdg_names

import seaborn as sns

In [ ]:
projects = {}
project_sdgs = {}

for fp in FRAMEWORK_PROGRAMMES:
    projects[fp] = load_cordis_projects(fp).set_index('rcn')
    project_sdgs[fp] = load_cordis_project_sdgs(fp, 'strict_label').set_index('rcn')

## Classification Coverage

In [ ]:
def coverage(projects, sdgs):
    c = sdgs.shape[0] / projects.shape[0] * 100
    return c

for fp in FRAMEWORK_PROGRAMMES:
    c = coverage(projects[fp], project_sdgs[fp])
    print(f'{c:.2f}% of {fp.upper()} projects were successfully classified')

In [ ]:
c = [coverage(projects[fp], project_sdgs[fp]) for fp in FRAMEWORK_PROGRAMMES]

fig, ax = plt.subplots()
ax.bar([fp.upper() for fp in FRAMEWORK_PROGRAMMES], c)
ax.set_ylabel('Classifier Coverage (%)')
ax.set_xlabel('Framework Programme');

## H2020, FP7 and FP6

### Aggregate Activity

In [ ]:
def plot_sdg_activity(sdg_df, ax, norm=False):
    '''plot_sdg_activity
    '''
    x = sdg_df.sum()
    if norm:
        x = x / x.sum()
    x.plot.barh(color=sdg_hex_color_codes().values(), ax=ax)
    ax.set_yticklabels(sdg_names().values())
    ax.invert_yaxis()

In [ ]:
fig, axs = plt.subplots(ncols=3, figsize=(15, 4), sharey=True)

plot_sdg_activity(project_sdgs['fp6'], axs[0])
axs[0].set_title('FP6')
plot_sdg_activity(project_sdgs['fp7'], axs[1])
axs[1].set_title('FP7')
plot_sdg_activity(project_sdgs['h2020'], axs[2])
axs[2].set_title('H2020');

In [ ]:
fig, axs = plt.subplots(ncols=3, figsize=(15, 4), sharey=True)

plot_sdg_activity(project_sdgs['fp6'], axs[0], norm=True)
axs[0].set_title('FP6')
plot_sdg_activity(project_sdgs['fp7'], axs[1], norm=True)
axs[1].set_title('FP7')
plot_sdg_activity(project_sdgs['h2020'], axs[2], norm=True)
axs[2].set_title('H2020');

In [ ]:
def plot_sdg_count(sdg_df, ax):
    '''plot_sdg_count
    '''
    sdg_df.sum(axis=1).value_counts().sort_index().plot.barh(ax=ax)
    ax.set_xlabel('N Projects')
    ax.set_ylabel('N SDGs')
    ax.invert_yaxis()

### Multiple Goal Frequency

In [ ]:
fig, axs = plt.subplots(ncols=3, figsize=(15, 4), sharey=True)

plot_sdg_count(project_sdgs['fp6'], axs[0])
axs[0].set_title('FP6')
plot_sdg_count(project_sdgs['fp7'], axs[1])
axs[1].set_title('FP7')
plot_sdg_count(project_sdgs['h2020'], axs[2])
axs[2].set_title('H2020');

### Combining H2020, FP7 and FP6

In [ ]:
sdg_df = pd.concat([project_sdgs[fp] for fp in FRAMEWORK_PROGRAMMES[:3]], axis=0)
project_df = pd.concat([projects[fp] for fp in FRAMEWORK_PROGRAMMES[:3]], axis=0)
project_df = project_df.merge(sdg_df, left_index=True, right_index=True, how='right')

In [ ]:
sdg_keys = list(range(1, 17))
sdg_keys_all = list(range(0, 17))

In [ ]:
project_df[0] = 0
project_df[0][project_df[sdg_keys].sum(axis=1) == 0] = 1

### Project Validation

In [ ]:
def validate(df, sdg):
    v = []
    rows = project_df[project_df[sdg] == 1].sample(10, random_state=0)
    print(f'=== {sdg_names().get(sdg, "No SDG")} ===')
    for i, row in rows.iterrows():
        title = row.title
        objective = row.objective
        print(f'>>> {title.upper()}')
        print(f'{objective}')
        x = input()
        v.append(x)
    return v

### Top Tf-Idf Terms

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from stop_words import get_stop_words

In [ ]:
stops = get_stop_words('en')
tfidf = TfidfVectorizer(min_df=10, max_df=0.3, ngram_range=(2,2), 
                        stop_words=stops, token_pattern=u'(?ui)\\b\\w*[a-z]+\\w*\\b')
tfidf_vecs = tfidf.fit_transform(project_df['objective'].fillna(''))

In [ ]:
def get_top_tfidf_scores(doc_ids, tfidf, tfidf_vecs, topn=20):
    tfidf_docs_sum = np.array(tfidf_vecs[doc_ids].sum(axis=1))[:, 0]
    top_term_ids = np.argsort(tfidf_docs_sum)[::-1][:topn]
    terms = np.array(tfidf.get_feature_names())[top_term_ids]
    return terms

In [ ]:
get_top_tfidf_scores((project_df[7] == 1).values, tfidf, tfidf_vecs)

### SDG Activity by Year

In [ ]:
def mask_by_year(df, date_col='start_date', start_year=2004, end_year=2019):
    '''mask_by_year
    
    Args:
        df (pd.DataFrame):
        date_col (str):
        start_year (int):
        end_year (int):
        
    Returns:
        (pd.DataFrame):
    '''
    return df[(df[date_col].dt.year >= start_year) & (df[date_col].dt.year <= end_year)]


def normalised_count_activity(df, groupby, value_columns, baseline_col):
    '''normalised_activity
    Calculates count of 
    '''
    total_activity_by_year = df.groupby(groupby)[baseline_col].count()
    normalised_activity = (df.groupby(groupby)[value_columns].sum()
                .divide(total_activity_by_year, axis=0)) * 100
    return normalised_activity


def plot_sdg_normalised_count_activity(project_df):
    opts = {
        'groupby': pd.Grouper(key='start_date', freq='Y'),
        'value_columns': list(range(1, 17)),
        'baseline_col': 'id',
    }
    normed = normalised_count_activity(project_df, **opts)
    fig, axs = plt.subplots(figsize=(7, 5))

    (normed
     .rename(columns=sdg_names())
     .plot(
         color=sdg_hex_color_codes().values(), 
         ax=axs, 
         legend=None, 
         alpha=.7, 
         linewidth=2)
    )
    axs.set_xlabel('Project Start Year')
    axs.set_ylabel('Share of Projects in Year (%)')
    axs.legend(loc='center left', bbox_to_anchor=(1, 0.5));

In [ ]:
start_year = 2004
end_year = 2019
project_df = mask_by_year(project_df, date_col='start_date', start_year=start_year, end_year=end_year)

plot_sdg_normalised_count_activity(project_df)

In [ ]:
fp_details = pd.DataFrame(
    {
        'name': FRAMEWORK_PROGRAMMES,
        'start_year': [2014, 2007, 2002, 1998, 1994, 1990, 1987, 1984],
        'end_year': [2020, 2013, 2006, 2002, 1998, 1994, 1991, 1987],
    })

In [ ]:
fig, ax = plt.subplots()
total_projects_by_year = project_df.groupby(pd.Grouper(key='start_date', freq='Y'))['id'].count()
total_projects_by_year.plot(ax=ax)
ax.axvline(
    pd.to_datetime(
        f"{fp_details.set_index('name').loc['h2020']['start_year']}-01-01"), 
        color='gray', 
        linestyle='--')
ax.axvline(
    pd.to_datetime(
        f"{fp_details.set_index('name').loc['fp7']['start_year']}-01-01"), 
        color='gray', 
        linestyle='--')

### Temporal Trends in SDG Quotient

In [ ]:
def create_quotient(X, binary=False):
    """Calculate the location quotient

    Divides the share of activity in a location by the share of activity in the UK total

    Args:
        X (pandas.DataFrame): DataFrame where rows are locations, columns are sectors 
            and values are activity in a given sector at a location.
        binary (bool, optional): If True, discretise the data with a cut-off value of 1

    Returns:
        pandas.DataFrame
    """
    Xm = X.values
    X = pd.DataFrame((Xm/Xm.sum(1)[:, np.newaxis])/(Xm.sum(0)/Xm.sum()),
            index=X.index, columns=X.columns)

    return (X > 1) if binary else X

In [ ]:
sdg_quotient_fp = create_quotient(project_df.groupby('framework_programme')[sdg_keys_all].sum())[sdg_keys]

In [ ]:
grad = sdg_quotient_fp.iloc[2] - sdg_quotient_fp.iloc[0]
order = grad.sort_values().index.values

In [ ]:
fig, ax = plt.subplots(figsize=(5, 10))
sdg_quotient_fp.iloc[::-1][order].T.plot.barh(
    ax=ax, color=['#014092', '#177afd', '#8bbdfe'], edgecolor='white')
ax.set_yticklabels([sdg_names()[i] for i in order])
ax.axvline(1, color='gray', linestyle='--')
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
ax.set_xlabel('Quotient');

### SDG Funding by Year

In [ ]:
start_year = 2004
end_year = 2019
project_df = project_df[(project_df['start_date'].dt.year >= start_year) &
                        (project_df['start_date'].dt.year <= end_year)]

start_date_1yr_grouper = pd.Grouper(key='start_date', freq='Y')

total_funding_by_year = project_df.groupby(start_date_1yr_grouper)['total_cost'].sum()
sdg_total_funding = project_df[sdg_keys_all] * project_df['total_cost'].values[:,None]
total_funding_by_sdg = sdg_total_funding.sum()
sdg_total_funding['start_date'] = project_df['start_date']

total_ec_funding_by_year = project_df.groupby(start_date_1yr_grouper)['ec_max_contribution'].sum()
sdg_total_ec_funding = project_df[sdg_keys_all] * project_df['ec_max_contribution'].values[:,None]
total_ec_funding_by_sdg = sdg_total_ec_funding.sum()
sdg_total_ec_funding['start_date'] = project_df['start_date']

In [ ]:
fund_share = (sdg_total_ec_funding.groupby(start_date_1yr_grouper)[sdg_keys].sum()
         .divide(total_ec_funding_by_year, axis=0)) * 100

fund_norm = (sdg_total_ec_funding.groupby(start_date_1yr_grouper)[sdg_keys].sum()
         .divide(total_ec_funding_by_sdg[sdg_keys], axis=1)) * 100

fig, axs = plt.subplots(ncols=2, figsize=(13.5, 5))

fund_share.rolling(3).mean().plot(color=sdg_hex_color_codes().values(), ax=axs[0], legend=None, alpha=.7, linewidth=2)
axs[0].set_xlabel('Project Start Year')
axs[0].set_ylabel('Share of Funding in Year (%)')

(fund_norm
 .rolling(3).mean()
 .rename(columns=sdg_names())
 .plot(color=sdg_hex_color_codes().values(), ax=axs[1], alpha=.7, linewidth=2))
axs[1].legend(loc='center left', bbox_to_anchor=(1, 0.5))
axs[1].set_xlabel('Project Start Year')
axs[1].set_ylabel('Share of Funding by Goal (%)');

### Temporal Trends in SDG Funding Quotient

In [ ]:
# start_date_5yr_grouper = pd.Grouper(key='start_date', freq='5Y')
sdg_fund_quotient_fp = create_quotient(
    sdg_total_ec_funding[sdg_keys_all].groupby(project_df['framework_programme'])[sdg_keys_all].sum())[sdg_keys]

In [ ]:
grad = sdg_fund_quotient_fp.iloc[2] - sdg_fund_quotient_fp.iloc[0]
order = grad.sort_values().index.values

In [ ]:
fig, ax = plt.subplots(figsize=(5, 10))
sdg_fund_quotient_fp.iloc[::-1][order].T.plot.barh(
    ax=ax, color=['#014092', '#177afd', '#8bbdfe'], edgecolor='white')
ax.set_yticklabels([sdg_names()[i] for i in order])
ax.axvline(1, color='gray', linestyle='--')
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
ax.set_xlabel('5 Year Funding Quotient');

### Coordinators

In [ ]:
import seaborn as sns

In [ ]:
coordinator_ohe = pd.get_dummies(project_df['coordinator_country'])
top_countries = coordinator_ohe.sum()[coordinator_ohe.sum() > 50].index

In [ ]:
project_count_by_country = project_df['coordinator_country'].value_counts()
project_share_by_country = (project_count_by_country / project_count_by_country.sum()) * 100
sdg_count_by_country = project_df.groupby('coordinator_country')[sdg_keys_all].sum()
sdg_share_by_country = (sdg_count_by_country / sdg_count_by_country.sum()) * 100

coordinator_sdg_lq = create_quotient(sdg_count_by_country)

In [ ]:
country = 'UK'

sdg_order = sdg_share_by_country.loc[country].sort_values(ascending=False).index.values
sdg_order = sdg_order[sdg_order != 0]

fig, axs = plt.subplots(figsize=(15, 7), ncols=4, nrows=4)

for sdg, ax in zip(sdg_order, axs.ravel()):
    top = sdg_share_by_country[sdg].sort_values(ascending=False)[:10]
    colors = ['C0' if c != country else 'C1' for c in top.index.values]
    top.plot.bar(ax=ax, color=colors, alpha=0.6)
    ax.set_title(sdg_names()[sdg])
    shares = project_share_by_country.loc[top.index.values]
    ax.scatter(shares.index.values, shares.values, color=colors)
    ax.set_xlabel('Coordinator Country')
    ax.set_ylabel('% of SDG')
    
plt.tight_layout();

### Coordinator Country Specialisation

#### Get EU Country Codes

In [ ]:
country_df = pd.read_json(f'{data_path}/raw/countries/countries_restcountries_api.json')

In [ ]:
europe = []
for code, c in zip(country_df['alpha2Code'], country_df['regionalBlocs']):
    for x in c:
        if x['acronym'] == 'EU':
            europe.append(code)
            
europe = sorted(['UK' if e == 'GB' else e for e in europe])

In [ ]:
eu_coordinator_sdg_lq = coordinator_sdg_lq.reindex(europe).dropna(how='all')

In [ ]:
eu_high = project_count_by_country[project_count_by_country > 50].reindex(europe).dropna().index.values

fig, axs = plt.subplots(figsize=(15, 7), ncols=4, nrows=4)

for sdg, ax in zip(sdg_keys, axs.ravel()):
    x = eu_coordinator_sdg_lq[sdg].loc[eu_high].sort_values(ascending=False)[:10]
    x.plot.bar(ax=ax, color=sdg_hex_color_codes()[sdg])
    ax.set_title(sdg_names()[sdg])
    ax.set_xlabel('Coordinator Country')
    ax.set_ylabel('LQ')
    ax.axhline(1, color='gray', linestyle='--')
    
plt.tight_layout();

Cyprus and Malta very high for water based.

In [ ]:
from sdg_mapping.geo.nuts import load_nuts_regions

In [ ]:
nuts = load_nuts_regions(2016, f'{data_path}/raw/shapefiles', level=0, resolution=60, countries=None)

In [ ]:
fig, axs = plt.subplots(ncols=4, nrows=4, figsize=(20,20))

for sdg, ax in zip(sdg_keys, axs.ravel()):

    nuts_sdg = nuts.set_index('CNTR_CODE')
    nuts_sdg['lq'] = eu_coordinator_sdg_lq[sdg].reindex(eu_high).fillna(0)
    nuts_sdg.plot(column='lq', ax=ax, cmap='viridis_r')
    ax.set_xlim(-15, 40)
    ax.set_ylim(30, 75)
    ax.set_title(sdg_names()[sdg])
    
plt.tight_layout();

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer

In [ ]:
mlb = MultiLabelBinarizer()
mlb_vecs = mlb.fit_transform(project_df['participant_countries'].fillna(''))

In [ ]:
y = pd.DataFrame(mlb_vecs, columns=mlb.classes_)

In [ ]:
y.sum()['DE']

In [ ]:
project_df['participant_countries']